# Starting Over

Python Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Custom Functions

In [2]:
from useful_functions.dd_dictionary import create_dd_dictionary
from useful_functions.pd_dictionary import create_pd_dictionary
from useful_functions.takeover_dataframe import create_takeover_timestamps

---

# Importing Data

### Order of obstacles 
- A=Deer
- B=Traffic Cone
- C=Frog
- D=Can
- E=False Alarm1
- F=False Alarm2

In [3]:
order_obstacle = pd.read_csv('../AdVitam/Exp2/order_obstacles.csv')
order_obstacle.head()

,Code,Obstacle 1,Obstacle 2,Obstacle 3,Obstacle 4,Obstacle 5,Obstacle 6
0,NST1,A,B,C,D,E,F
1,ST2,B,A,F,E,C,D
2,NST3,C,F,B,A,D,E
3,ST4,D,C,E,B,F,A
4,NST5,E,D,A,F,B,C


### Driving Data

Metadata:
- Time = Time elapsed since the software was launched (in seconds)
- EngineSpeed = Engine speed (in rpm)
- GearPosActual = Current gear
- GearPosTarget = Next planned gear
- AcceleratorPedalPos = Position of gas pedal. Should not be used (see below)
- DeceleratorPedalPos = Position of brake pedal. Should not be used (see below)
- SteeringWheelAngle = Steering wheel angle (in degrees)
- VehicleSpeed = Vehicle speed (in mph)
- Position X = Vehicle position along the x-axis in the simulated driving environment
- Position Y = Vehicle position along the y-axis in the simulated driving environment
- Position Z = Vehicle position along the z-axis in the simulated driving environment
- Autonomous Mode (T/F) = Autonomous pilot status. True = autonomous pilot activated, False = autonomous pilot deactivated (driver in control of the car)
- Obstacles: Events that occurred during the driving simulation. 
	- TriggeredObsX = Time at which each takeover request was triggered by the experimenter.. 
	- Obs1 = deer, Obs2 = traffic cone, Obs3 = frog, Obs4 = traffic cone, Obs5 = false alarm (x2). 
	- Detected = Time at which the driver pressed the steering wheel button to notify he/she understood the situation. The driver is in control of the car when the value of the column "Autonomous Mode (T/F)" is False.

Note: Due to recording problem, the "AcceleratorPedalPos" and "DeceleratorPedalPos" columns do not correspond to the gas and brake pedal position.

In [4]:
driving_data_dictionary = create_dd_dictionary("../AdVitam/Exp2/Raw/Driving")
driving_data_dictionary.keys()

dict_keys(['NST41', 'ST20', 'NST55', 'ST34', 'NST69', 'ST08', 'NST83', 'NST81', 'ST22', 'NST43', 'ST36', 'NST57', 'NST53', 'ST32', 'NST47', 'ST26', 'NST85', 'NST91', 'NST87', 'ST30', 'NST51', 'ST24', 'NST45', 'ST18', 'NST79', 'ST80', 'ST56', 'NST37', 'ST42', 'NST23', 'NST35', 'ST54', 'NST21', 'ST40', 'NST09', 'ST68', 'ST82', 'ST86', 'ST44', 'NST25', 'ST50', 'NST31', 'ST78', 'NST19', 'NST27', 'ST46', 'NST33', 'ST52', 'ST90', 'ST84', 'NST03', 'ST62', 'NST17', 'ST76', 'ST88', 'ST48', 'NST29', 'ST60', 'NST01', 'ST74', 'NST15', 'NST39', 'ST58', 'NST11', 'ST70', 'NST05', 'ST64', 'ST72', 'NST13', 'ST66', 'NST07', 'ST28', 'NST49', 'ST14', 'NST75', 'NST61', 'NST77', 'ST16', 'NST63', 'ST02', 'NST89', 'ST06', 'NST67', 'ST12', 'NST73', 'NST59', 'ST38', 'NST65', 'ST04', 'NST71', 'ST10'])

### Physiological Signals

Metadata:
- min = Time elaspsed 
- ECG = Electrocardiogram (1000Hz)
- EDA = Electrodermal Activity (1000Hz)
- RESP = Resperatory (1000Hz)

### Physiological Signal Markers
contains the timestamps for each period of the experiment. 

- Training1 = Baseline phase
- Training2 = Practice phase in the driving simulator
- Driving = Main driving session in conditionally automated driving. 

Be careful, the timestamps are here in seconds while they are in minutes in the raw data.


In [5]:
phsyiological_data = create_pd_dictionary('../AdVitam/Exp2/Raw/Physio/Txt')
phsyiological_data.keys()

dict_keys(['NST89-markers.txt', 'NST41.txt', 'ST20.txt', 'NST55.txt', 'ST34.txt', 'NST69.txt', 'ST08.txt', 'NST83.txt', 'ST80-markers.txt', 'NST23-markers.txt', 'ST18-markers.txt', 'NST53-markers.txt', 'ST68-markers.txt', 'ST36-markers.txt', 'NST81.txt', 'ST46-markers.txt', 'ST22.txt', 'NST43.txt', 'ST36.txt', 'NST57.txt', 'NST61-markers.txt', 'NST11-markers.txt', 'ST74-markers.txt', 'ST04-markers.txt', 'NST85-markers.txt', 'ST90-markers.txt', 'NST53.txt', 'ST32.txt', 'NST47.txt', 'ST26.txt', 'NST85.txt', 'NST01-markers.txt', 'NST71-markers.txt', 'ST14-markers.txt', 'NST87.txt', 'ST64-markers.txt', 'ST30.txt', 'NST51.txt', 'ST24.txt', 'NST45.txt', 'ST78-markers.txt', 'ST18.txt', 'NST79.txt', 'NST43-markers.txt', 'ST08-markers.txt', 'NST33-markers.txt', 'ST56-markers.txt', 'ST26-markers.txt', 'ST86-markers.txt', 'ST80.txt', 'ST56.txt', 'NST37.txt', 'ST42.txt', 'NST23.txt', 'NST17-markers.txt', 'NST67-markers.txt', 'ST02-markers.txt', 'NST39-markers.txt', 'ST72-markers.txt', 'NST35.txt',

### Driver Demographic Data

In [7]:
driver_demographic_data = pd.read_csv('../AdVitam/Exp2/Preprocessed/Questionnaires/Exp2_Database.csv')
driver_demographic_data.head()

,code,date,time,condition,sex,age,mothertongue,education,driving_license,km_year,...,sart_6_fa,sart_7_fa,sart_8_fa,sart_10_fa,demand_fa,supply_fa,understanding_fa,sart_global_fa,nb_times_remind_counting,notes
0,NST1,51218,90503,1,1,19,1,1,2017,200,...,6,7,5,7,8,27,16,35,NaN,I accidentally triggered the F9 obstacle twice...
1,ST2,51218,100823,2,1,19,1,1,2017,5000,...,1,2,1,7,3,6,12,15,1.0,NaN
2,NST3,51218,161024,1,1,19,1,1,2017,1000,...,3,6,1,2,3,17,7,21,NaN,"Timestamp driving start : Driving End, the 1st..."
3,ST4,61218,144813,2,1,21,3,2,2016,1500,...,2,4,2,2,6,12,8,14,1.0,"In the testing phase 1, we have to remove the ..."
4,NST5,61218,162147,1,1,22,1,1,2017,1500,...,4,2,4,1,3,18,9,24,NaN,NaN


---

# Defining Takeover Quality Quantitatively

- Takeover Time (TOT)
- Sudden Vehicle Deviation
- Response Budget

### Takeover Time 

In [6]:
takeover_timestamps = create_takeover_timestamps(driving_data_dictionary)
takeover_timestamps.head()

,TriggeredObs1,TakeoverObs1,ReleaseObs1,TOTObs1,TriggeredObs2,TakeoverObs2,ReleaseObs2,TOTObs2,TriggeredObs3,TakeoverObs3,ReleaseObs3,TOTObs3,TriggeredObs4,TakeoverObs4,ReleaseObs4,TOTObs4,TriggeredObs5,TakeoverObs5,ReleaseObs5,TOTObs5
NST41,601.3568,605.3909,629.1172,4.0341,953.3417,955.9106,965.5509,2.5689,1191.0560,1210.1620,1217.8050,19.1060,386.3657,388.9877,399.9649,2.6220,257.8800,388.9877,399.9649,131.1077
ST20,391.2977,394.5964,401.4673,3.2987,258.5524,260.7675,267.6068,2.2151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NST55,292.0465,299.4751,306.1428,7.4286,458.2091,461.9518,467.2733,3.7427,606.1623,717.9875,721.6473,111.8252,715.2051,717.9875,721.6473,2.7824,NaN,NaN,NaN,NaN
ST34,1207.3810,1213.6300,1221.8050,6.2490,776.5834,779.0553,784.4412,2.4719,450.1733,453.5964,460.6361,3.4231,283.8758,288.0347,290.1353,4.1589,617.4570,779.0553,784.4412,161.5983
NST69,723.2592,726.9050,733.1387,3.6458,534.8524,536.3959,543.8203,1.5435,227.8289,230.7167,235.5535,2.8878,929.3593,931.6457,941.9799,2.2864,360.0632,536.3959,543.8203,176.3327


# !! Here!!

Why are the results I calculated different than the ones in the preprocessed data?
